In [1]:
#import tensorflow.lite as tflite
import tensorflow as tf
from tensorflow import keras
import tflite_runtime.interpreter as tflite
from io import BytesIO
from urllib import request
import numpy as np

from PIL import Image

2023-11-24 16:51:42.725232: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 16:51:43.580703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 16:51:43.581907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 16:51:43.750284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 16:51:44.092880: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 16:51:44.096727: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
model = keras.models.load_model("bees-wasps.h5")

**Question 1**

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp4szoqljh/assets


INFO:tensorflow:Assets written to: /tmp/tmp4szoqljh/assets
2023-11-24 16:51:50.972821: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-24 16:51:50.972883: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-24 16:51:50.973529: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4szoqljh
2023-11-24 16:51:50.974641: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-24 16:51:50.974658: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp4szoqljh
2023-11-24 16:51:50.984352: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-11-24 16:51:50.984986: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-24 16:51:51.347451: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

*Answer:* 43Mb

**Question 2**

In [4]:
interpreter = tflite.Interpreter(model_path="bees-wasps.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
output_index

13

*Answer:* 13

In [6]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_input(x):
    x /= 255
    return x

In [7]:
image = download_image("https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg")
input_size = (150, 150)

img = prepare_image(image, input_size)

**Question 3:**

In [8]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [9]:
X

array([[[[0.94509804, 0.9098039 , 0.8509804 ],
         [0.9372549 , 0.92156863, 0.9764706 ],
         [0.9137255 , 0.8980392 , 0.95686275],
         ...,
         [0.2901961 , 0.33333334, 0.16470589],
         [0.34901962, 0.40784314, 0.15294118],
         [0.29803923, 0.36078432, 0.11764706]],

        [[0.9490196 , 0.9098039 , 0.8627451 ],
         [0.91764706, 0.9098039 , 0.9607843 ],
         [0.9019608 , 0.89411765, 0.9490196 ],
         ...,
         [0.27450982, 0.3372549 , 0.16078432],
         [0.4627451 , 0.5058824 , 0.19607843],
         [0.4509804 , 0.49411765, 0.18431373]],

        [[0.92941177, 0.88235295, 0.81960785],
         [0.9137255 , 0.9098039 , 0.9647059 ],
         [0.90588236, 0.89411765, 0.9607843 ],
         ...,
         [0.32156864, 0.3764706 , 0.1764706 ],
         [0.49803922, 0.5294118 , 0.20392157],
         [0.43137255, 0.46666667, 0.21176471]],

        ...,

        [[0.03137255, 0.06666667, 0.04705882],
         [0.02352941, 0.08627451, 0.04705882]

*Answer:* 0.9450980

**Question 4:**

In [10]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [11]:
preds

array([[0.6592137]], dtype=float32)

*Answer:* 0.658

**Question 5**

*Answer:* 662Mb

**Question 6**

*Answer:* 0.4458